In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader 
import timm
from torchvision import transforms
import os
from PIL import Image
import pandas as pd
from training_utils import *
from sklearn.metrics import accuracy_score, roc_auc_score, fbeta_score, precision_score, recall_score, confusion_matrix

/home/giuto/Desktop/Uni/Magistrale/1° Anno/II° Semestre/Computer vision/Murgia-AI-Wildlife-Track/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_model(model, dataloader, device, label_map):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Ricava classi presenti effettivamente nel test
    unique_labels = sorted(list(set(all_labels + all_preds)))
    inv_label_map = {v: k for k, v in label_map.items()}
    class_names = [inv_label_map[i] for i in unique_labels]

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, labels=unique_labels, target_names=class_names))

    print("\nConfusion Matrix:")
    cm = confusion_matrix(all_labels, all_preds, labels=unique_labels)
    print(cm)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Trasformazioni immagine per ViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
label_map = {
    'badger': 0,
    'bird': 1,
    'boar': 2,
    'butterfly': 3,
    'cat': 4,
    'dog': 5,
    'fox': 6,
    'lizard': 7,
    'podolic_cow': 8,
    'porcupine': 9,
    'weasel': 10,
    'wolf': 11
}

In [6]:
# Dataset e DataLoader
train_ds = AnimalDataset("data/train.csv", "data/labeled_img/", transform=transform, label_map=label_map, crop_bbox=True)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

val_ds = AnimalDataset("data/val.csv", "data/labeled_img/", transform=transform, label_map=label_map, crop_bbox=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

test_ds = AnimalDataset("data/test.csv", "data/labeled_img/", transform=transform, label_map=label_map, crop_bbox=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

### Trining of Vit model with frozen backbone

In [7]:
num_classes = len(train_ds.label_map)

# --- Backbone congelato (solo testa allenata) ---
model_frozen = timm.create_model('vit_base_patch16_224', pretrained=True)
model_frozen.head = nn.Linear(model_frozen.head.in_features, num_classes)  # sostituisci testa

# Congela tutti i parametri
for param in model_frozen.parameters():
    param.requires_grad = False
# Sblocca solo la testa
for param in model_frozen.head.parameters():
    param.requires_grad = True

model_frozen = model_frozen.to(device)

optimizer_frozen = torch.optim.Adam(filter(lambda p: p.requires_grad, model_frozen.parameters()), lr=1e-3)
criterion_frozen = nn.CrossEntropyLoss()

In [ ]:
# Loop di training completo
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model_frozen, train_loader, optimizer_frozen, criterion_frozen, device)
    val_loss, val_acc = eval_model(model_frozen, val_loader, criterion_frozen, device)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")

test_loss, test_acc = eval_model(model_frozen, test_loader, criterion_frozen, device)
print(f"\n🧪 Test finale → Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")

# 📈 Metriche complete sul test set
print("\n📋 Metriche di classificazione dettagliate:")
evaluate_model(model_frozen, test_loader, device, train_ds.label_map)

Epoch 1/5 - Train loss: 0.8941, acc: 0.7889 | Val loss: 0.4864, acc: 0.8614
Epoch 2/5 - Train loss: 0.2351, acc: 0.9254 | Val loss: 0.3619, acc: 0.8911
Epoch 3/5 - Train loss: 0.1319, acc: 0.9595 | Val loss: 0.3122, acc: 0.8713
Epoch 4/5 - Train loss: 0.0823, acc: 0.9808 | Val loss: 0.2990, acc: 0.9010
Epoch 5/5 - Train loss: 0.0526, acc: 0.9936 | Val loss: 0.2919, acc: 0.9109

🧪 Test finale → Loss: 0.0746 | Accuracy: 0.9901


### Trining of Vit model updating all weight

In [ ]:
# --- Fine tuning completo (tutti i parametri allenabili) ---
model_finetune = timm.create_model('vit_base_patch16_224', pretrained=True)
model_finetune.head = nn.Linear(model_finetune.head.in_features, num_classes)

# Sblocca tutti i parametri
for param in model_finetune.parameters():
    param.requires_grad = True

model_finetune = model_finetune.to(device)

optimizer_finetune = torch.optim.Adam(model_finetune.parameters(), lr=1e-4)
criterion_finetune = nn.CrossEntropyLoss()

In [ ]:
# Loop di training completo
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model_finetune, train_loader, optimizer_finetune, criterion_finetune, device)
    val_loss, val_acc = eval_model(model_finetune, val_loader, criterion_finetune, device)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")

# 📊 Valutazione finale sul test set
test_loss, test_acc = eval_model(model_finetune, test_loader, criterion_finetune, device)
print(f"\n🧪 Test finale → Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")

# 📈 Metriche complete sul test set
print("\n📋 Metriche di classificazione dettagliate:")
evaluate_model(model_finetune, test_loader, device, train_ds.label_map)

Epoch 1/5 - Train loss: 0.7229, acc: 0.8017 | Val loss: 0.5959, acc: 0.8218
Epoch 2/5 - Train loss: 0.3142, acc: 0.8998 | Val loss: 0.5023, acc: 0.8317
Epoch 3/5 - Train loss: 0.1557, acc: 0.9552 | Val loss: 1.4990, acc: 0.7525
Epoch 4/5 - Train loss: 0.1310, acc: 0.9510 | Val loss: 0.5365, acc: 0.8911
Epoch 5/5 - Train loss: 0.0804, acc: 0.9723 | Val loss: 0.6674, acc: 0.8416

🧪 Test finale → Loss: 0.1693 | Accuracy: 0.9604

📋 Metriche di classificazione dettagliate:

Classification Report:
              precision    recall  f1-score   support

        bird       1.00      1.00      1.00        31
        boar       0.94      0.94      0.94        17
         cat       1.00      1.00      1.00         1
         fox       0.75      1.00      0.86         6
 podolic_cow       1.00      0.93      0.97        46
        wolf       0.00      0.00      0.00         0

    accuracy                           0.96       101
   macro avg       0.78      0.81      0.79       101
weighted avg   

/home/giuto/Desktop/Uni/Magistrale/1° Anno/II° Semestre/Computer vision/Murgia-AI-Wildlife-Track/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/giuto/Desktop/Uni/Magistrale/1° Anno/II° Semestre/Computer vision/Murgia-AI-Wildlife-Track/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/giuto/Desktop/Uni/Magistrale/1° Anno/II° Semestre/Computer vision/Murgia-AI-Wildlife-Track/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Reca